# Census- Employment Status Data

In [2]:
# Make sure you know where your folder you are at to save csv

In [2]:
import pandas as pd
import requests

In [3]:
# Census Subject Table API for Employment Status data within Elementary School Districts in California for 2015
url="https://api.census.gov/data/2015/acs/acs1/subject?get=group(S2301)&for=school%20district%20(elementary)&in=state:06"

In [4]:
# Request for HTTP Data from Census API, which is working <Response [200]>
response = requests.get(url)

In [5]:
# Resetting data from API Data for future formatting, lists data in one column
response_json = response.json()

In [6]:
# Places data in a dataframe and drops index column 0 (with headers), which has 51 schools and 702 columns of variables
elementaryschool_df = pd.DataFrame(response_json,columns=response_json[0]).drop(0)

In [3]:
# Print dataframe 
# elementaryschool_df

In [4]:
# Save dataframe to csv
# elementaryschool_df.to_csv("/Users/nataligracia/git/ca-school-enrollment-trend/Elementary2015test.csv")

In [5]:
# View School District to view column titles, 2015 data does not include district name
# elementaryschool_df.loc[elementaryschool_df["NAME"]=="Los Angeles Unified School District, California"]

## _Format Data_

In [7]:
# Census Subject Table Variables for Employment Status data
variableurl = "https://api.census.gov/data/2015/acs/acs1/subject/variables.json"

In [8]:
# Request for HTTP Data from Census API and reset data
variables_json = requests.get(variableurl).json()

In [9]:
# View variable for column title
variables_json["variables"]["S2301_C01_001E"]

{'label': 'Total!!Estimate!!Population 16 years and over',
 'concept': 'EMPLOYMENT STATUS',
 'predicateType': 'int',
 'group': 'S2301',
 'limit': 0,
 'attributes': 'S2301_C01_001M,S2301_C01_001EA'}

In [7]:
# Find and replace all columns with variable titles
new_labels = []
for col in elementaryschool_df.columns:
    label = variables_json["variables"].get(col)
    if label is not None:
        label = label['label']
    else:
        if col[-2:] == 'EA':
            label = variables_json["variables"].get(col[:-1])
            label = label['label'] + "||Annotation"
        elif col[-1] == 'M':
            label = variables_json["variables"].get(col[:-1]+'E')
            label = label['label'] + "||MarginOfError"
        elif col[-2:] == 'MA':
            label = variables_json["variables"].get(col[:-2]+'E')
            label = label['label'] + "||MarginOfErrorAnnotation"
    new_labels.append(label)

NameError: name 'elementaryschool_df' is not defined

In [6]:
# Find any columns without titles
new_labels  

In [12]:
# Change column titles for columns labeled "None" 
assert len(new_labels) == len(elementaryschool_df.columns)

In [13]:
# Confirm the number of columns without titles
sum([1 for x in new_labels if x is None])

2

In [14]:
# Setup new Labels of columns labeled "None"
# new_labels[-283] = 'NAME'
new_labels[-2] = 'STATE'
new_labels[-1] = 'SCHOOL DISTRICT (ELEMENTARY)'

In [15]:
# Create new labels of columns labeled "None"
elementaryschool_df.columns = new_labels

In [8]:
# Find all columns without "Annotation" in column title
# [col for col in elementaryschool_df.columns if "Annotation" not in col]

In [17]:
# Create a new dataframe for data without the columns that have "Annotation" in the title, which is 702 columns
without_annotation = elementaryschool_df[[col for col in elementaryschool_df.columns if "Annotation" not in col]].copy()

In [18]:
without_annotation.head()

,Total!!Estimate!!Population 16 years and over,Total!!Estimate!!Population 16 years and over||MarginOfError,Total!!Estimate!!Population 16 years and over||MarginOfError,Total!!Estimate!!AGE!!16 to 19 years,Total!!Estimate!!AGE!!16 to 19 years||MarginOfError,Total!!Estimate!!AGE!!16 to 19 years||MarginOfError,Total!!Estimate!!AGE!!20 to 24 years,Total!!Estimate!!AGE!!20 to 24 years||MarginOfError,Total!!Estimate!!AGE!!20 to 24 years||MarginOfError,Total!!Estimate!!AGE!!25 to 29 years,...,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Less than high school graduate||MarginOfError,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Less than high school graduate||MarginOfError,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!High school graduate (includes equivalency)||MarginOfError,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!High school graduate (includes equivalency)||MarginOfError,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Some college or associate's degree||MarginOfError,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Some college or associate's degree||MarginOfError,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Bachelor's degree or higher||MarginOfError,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Bachelor's degree or higher||MarginOfError,STATE,SCHOOL DISTRICT (ELEMENTARY)
1,94280,5145,5145,7413,1477,1477,10469,1644,1644,8790,...,2.6,2.6,3.9,3.9,2.7,2.7,2.2,2.2,06,02310
2,161041,6674,6674,11112,1210,1210,17032,2091,2091,19337,...,2.4,2.4,2.7,2.7,2.8,2.8,2.5,2.5,06,02610
3,146865,8341,8341,13334,1888,1888,16341,2745,2745,16323,...,4.7,4.7,4.1,4.1,4.2,4.2,3.6,3.6,06,03630
4,64015,5163,5163,2735,772,772,4767,1332,1332,6492,...,10.2,10.2,3.4,3.4,4.3,4.3,3,3,06,04800
5,74714,2578,2578,6671,929,929,8885,1483,1483,6438,...,2,2,4.9,4.9,3.4,3.4,3.7,3.7,06,05580


In [19]:
# Find all columns without "MarginOfError" in column title
# [col for col in without_annotation_df.columns if "MarginOfError" not in col]

In [20]:
# Create a new dataframe for data without the columns that have "MarginOfError" in the title, which is 142 columns
withoutmarginerror = without_annotation[[col for col in without_annotation.columns if 'MarginOfError' not in col]].copy()

In [21]:
withoutmarginerror.head()

,Total!!Estimate!!Population 16 years and over,Total!!Estimate!!AGE!!16 to 19 years,Total!!Estimate!!AGE!!20 to 24 years,Total!!Estimate!!AGE!!25 to 29 years,Total!!Estimate!!AGE!!30 to 34 years,Total!!Estimate!!AGE!!35 to 44 years,Total!!Estimate!!AGE!!45 to 54 years,Total!!Estimate!!AGE!!55 to 59 years,Total!!Estimate!!AGE!!60 to 64 years,Total!!Estimate!!AGE!!65 to 74 years,...,Unemployment rate!!Estimate!!POVERTY STATUS IN THE PAST 12 MONTHS!!Below poverty level,Unemployment rate!!Estimate!!POVERTY STATUS IN THE PAST 12 MONTHS!!At or above the poverty level,Unemployment rate!!Estimate!!DISABILITY STATUS!!With any disability,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Less than high school graduate,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!High school graduate (includes equivalency),Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Some college or associate's degree,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Bachelor's degree or higher,STATE,SCHOOL DISTRICT (ELEMENTARY)
1,94280,7413,10469,8790,7924,16735,15987,6619,6101,7826,...,23.6,6.4,10.9,6.2,6.6,8.4,4.7,4.5,06,02310
2,161041,11112,17032,19337,16624,33681,27585,9667,7068,10698,...,16,5.1,10.6,5.7,6.2,6.4,5.7,3.9,06,02610
3,146865,13334,16341,16323,16582,24384,22204,9264,8391,13685,...,24.5,10.4,32.4,13.2,16.2,14,11.6,7,06,03630
4,64015,2735,4767,6492,6335,9895,11440,5638,5537,6946,...,35.6,4.7,12.1,6.3,11.6,3.3,5.7,6.4,06,04800
5,74714,6671,8885,6438,9792,15099,9882,4613,3847,5782,...,18.7,4.8,26.3,5.1,4.1,7.4,7.7,0,06,05580


In [22]:
# Find all columns without "Labor Force Participation Rate" in column title
# [col for col in withoutmarginerror.columns if "Labor Force Participation Rate" not in col]

In [23]:
# Create a new dataframe for data without the columns that have "Labor Force Participation Rate" in the title, which is 107 columns
withoutlaborforce = withoutmarginerror[[col for col in withoutmarginerror.columns if 'Labor Force Participation Rate' not in col]].copy()

In [24]:
withoutlaborforce.head()

,Total!!Estimate!!Population 16 years and over,Total!!Estimate!!AGE!!16 to 19 years,Total!!Estimate!!AGE!!20 to 24 years,Total!!Estimate!!AGE!!25 to 29 years,Total!!Estimate!!AGE!!30 to 34 years,Total!!Estimate!!AGE!!35 to 44 years,Total!!Estimate!!AGE!!45 to 54 years,Total!!Estimate!!AGE!!55 to 59 years,Total!!Estimate!!AGE!!60 to 64 years,Total!!Estimate!!AGE!!65 to 74 years,...,Unemployment rate!!Estimate!!POVERTY STATUS IN THE PAST 12 MONTHS!!Below poverty level,Unemployment rate!!Estimate!!POVERTY STATUS IN THE PAST 12 MONTHS!!At or above the poverty level,Unemployment rate!!Estimate!!DISABILITY STATUS!!With any disability,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Less than high school graduate,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!High school graduate (includes equivalency),Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Some college or associate's degree,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Bachelor's degree or higher,STATE,SCHOOL DISTRICT (ELEMENTARY)
1,94280,7413,10469,8790,7924,16735,15987,6619,6101,7826,...,23.6,6.4,10.9,6.2,6.6,8.4,4.7,4.5,06,02310
2,161041,11112,17032,19337,16624,33681,27585,9667,7068,10698,...,16,5.1,10.6,5.7,6.2,6.4,5.7,3.9,06,02610
3,146865,13334,16341,16323,16582,24384,22204,9264,8391,13685,...,24.5,10.4,32.4,13.2,16.2,14,11.6,7,06,03630
4,64015,2735,4767,6492,6335,9895,11440,5638,5537,6946,...,35.6,4.7,12.1,6.3,11.6,3.3,5.7,6.4,06,04800
5,74714,6671,8885,6438,9792,15099,9882,4613,3847,5782,...,18.7,4.8,26.3,5.1,4.1,7.4,7.7,0,06,05580


In [25]:
# Find all columns without "Sex" in column title
# [col for col in withoutlaborforce.columns if "SEX" not in col]

In [26]:
# Create a new dataframe for data without the columns that have "Sex" in the title, which is 89 columns
withoutsex = withoutlaborforce[[col for col in withoutlaborforce.columns if 'SEX' not in col]].copy()

In [27]:
withoutsex.head()

,Total!!Estimate!!Population 16 years and over,Total!!Estimate!!AGE!!16 to 19 years,Total!!Estimate!!AGE!!20 to 24 years,Total!!Estimate!!AGE!!25 to 29 years,Total!!Estimate!!AGE!!30 to 34 years,Total!!Estimate!!AGE!!35 to 44 years,Total!!Estimate!!AGE!!45 to 54 years,Total!!Estimate!!AGE!!55 to 59 years,Total!!Estimate!!AGE!!60 to 64 years,Total!!Estimate!!AGE!!65 to 74 years,...,Unemployment rate!!Estimate!!POVERTY STATUS IN THE PAST 12 MONTHS!!Below poverty level,Unemployment rate!!Estimate!!POVERTY STATUS IN THE PAST 12 MONTHS!!At or above the poverty level,Unemployment rate!!Estimate!!DISABILITY STATUS!!With any disability,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Less than high school graduate,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!High school graduate (includes equivalency),Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Some college or associate's degree,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Bachelor's degree or higher,STATE,SCHOOL DISTRICT (ELEMENTARY)
1,94280,7413,10469,8790,7924,16735,15987,6619,6101,7826,...,23.6,6.4,10.9,6.2,6.6,8.4,4.7,4.5,06,02310
2,161041,11112,17032,19337,16624,33681,27585,9667,7068,10698,...,16,5.1,10.6,5.7,6.2,6.4,5.7,3.9,06,02610
3,146865,13334,16341,16323,16582,24384,22204,9264,8391,13685,...,24.5,10.4,32.4,13.2,16.2,14,11.6,7,06,03630
4,64015,2735,4767,6492,6335,9895,11440,5638,5537,6946,...,35.6,4.7,12.1,6.3,11.6,3.3,5.7,6.4,06,04800
5,74714,6671,8885,6438,9792,15099,9882,4613,3847,5782,...,18.7,4.8,26.3,5.1,4.1,7.4,7.7,0,06,05580


In [28]:
# Find all columns without "Poverty Status" in column title
# [col for col in withoutsex.columns if "POVERTY STATUS" not in col]

In [29]:
# Create a new dataframe for data without the columns that have "Poverty Status" in the title, which is 83 columns
withoutps = withoutsex[[col for col in withoutsex.columns if 'POVERTY STATUS' not in col]].copy()

In [30]:
withoutps.head()

,Total!!Estimate!!Population 16 years and over,Total!!Estimate!!AGE!!16 to 19 years,Total!!Estimate!!AGE!!20 to 24 years,Total!!Estimate!!AGE!!25 to 29 years,Total!!Estimate!!AGE!!30 to 34 years,Total!!Estimate!!AGE!!35 to 44 years,Total!!Estimate!!AGE!!45 to 54 years,Total!!Estimate!!AGE!!55 to 59 years,Total!!Estimate!!AGE!!60 to 64 years,Total!!Estimate!!AGE!!65 to 74 years,...,"Unemployment rate!!Estimate!!White alone, not Hispanic or Latino",Unemployment rate!!Estimate!!Population 20 to 64 years,Unemployment rate!!Estimate!!DISABILITY STATUS!!With any disability,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Less than high school graduate,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!High school graduate (includes equivalency),Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Some college or associate's degree,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Bachelor's degree or higher,STATE,SCHOOL DISTRICT (ELEMENTARY)
1,94280,7413,10469,8790,7924,16735,15987,6619,6101,7826,...,6.4,7.6,10.9,6.2,6.6,8.4,4.7,4.5,06,02310
2,161041,11112,17032,19337,16624,33681,27585,9667,7068,10698,...,4.2,6.1,10.6,5.7,6.2,6.4,5.7,3.9,06,02610
3,146865,13334,16341,16323,16582,24384,22204,9264,8391,13685,...,9.5,13.2,32.4,13.2,16.2,14,11.6,7,06,03630
4,64015,2735,4767,6492,6335,9895,11440,5638,5537,6946,...,2,6.4,12.1,6.3,11.6,3.3,5.7,6.4,06,04800
5,74714,6671,8885,6438,9792,15099,9882,4613,3847,5782,...,9.3,6.6,26.3,5.1,4.1,7.4,7.7,0,06,05580


In [31]:
# Find all columns without "Disability Status" in column title
# [col for col in withoutps.columns if "DISABILITY STATUS" not in col]

In [32]:
# Create a new dataframe for data without the columns that have "Disability Status" in the title, which is 80 columns
withoutds = withoutps[[col for col in withoutps.columns if 'DISABILITY STATUS' not in col]].copy()

In [35]:
withoutds.head()

,Total!!Estimate!!Population 16 years and over,Total!!Estimate!!AGE!!16 to 19 years,Total!!Estimate!!AGE!!20 to 24 years,Total!!Estimate!!AGE!!25 to 29 years,Total!!Estimate!!AGE!!30 to 34 years,Total!!Estimate!!AGE!!35 to 44 years,Total!!Estimate!!AGE!!45 to 54 years,Total!!Estimate!!AGE!!55 to 59 years,Total!!Estimate!!AGE!!60 to 64 years,Total!!Estimate!!AGE!!65 to 74 years,...,Unemployment rate!!Estimate!!Hispanic or Latino origin (of any race),"Unemployment rate!!Estimate!!White alone, not Hispanic or Latino",Unemployment rate!!Estimate!!Population 20 to 64 years,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Less than high school graduate,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!High school graduate (includes equivalency),Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Some college or associate's degree,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Bachelor's degree or higher,STATE,SCHOOL DISTRICT (ELEMENTARY)
1,94280,7413,10469,8790,7924,16735,15987,6619,6101,7826,...,8.8,6.4,7.6,6.2,6.6,8.4,4.7,4.5,06,02310
2,161041,11112,17032,19337,16624,33681,27585,9667,7068,10698,...,7.6,4.2,6.1,5.7,6.2,6.4,5.7,3.9,06,02610
3,146865,13334,16341,16323,16582,24384,22204,9264,8391,13685,...,15.2,9.5,13.2,13.2,16.2,14,11.6,7,06,03630
4,64015,2735,4767,6492,6335,9895,11440,5638,5537,6946,...,-999999999,2,6.4,6.3,11.6,3.3,5.7,6.4,06,04800
5,74714,6671,8885,6438,9792,15099,9882,4613,3847,5782,...,6,9.3,6.6,5.1,4.1,7.4,7.7,0,06,05580


In [36]:
# Find all columns without "Educational Attainment" in column title
# [col for col in withoutds.columns if "EDUCATIONAL ATTAINMENT" not in col]

In [37]:
# Create a new dataframe for data without the columns that have "Educational Attainment" in the title, which is 65 columns
withoutea = withoutds[[col for col in withoutds.columns if 'EDUCATIONAL ATTAINMENT' not in col]].copy()

In [38]:
withoutea.head()

,Total!!Estimate!!Population 16 years and over,Total!!Estimate!!AGE!!16 to 19 years,Total!!Estimate!!AGE!!20 to 24 years,Total!!Estimate!!AGE!!25 to 29 years,Total!!Estimate!!AGE!!30 to 34 years,Total!!Estimate!!AGE!!35 to 44 years,Total!!Estimate!!AGE!!45 to 54 years,Total!!Estimate!!AGE!!55 to 59 years,Total!!Estimate!!AGE!!60 to 64 years,Total!!Estimate!!AGE!!65 to 74 years,...,Unemployment rate!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!American Indian and Alaska Native alone,Unemployment rate!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Asian alone,Unemployment rate!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Native Hawaiian and Other Pacific Islander alone,Unemployment rate!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Some other race alone,Unemployment rate!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Two or more races,Unemployment rate!!Estimate!!Hispanic or Latino origin (of any race),"Unemployment rate!!Estimate!!White alone, not Hispanic or Latino",Unemployment rate!!Estimate!!Population 20 to 64 years,STATE,SCHOOL DISTRICT (ELEMENTARY)
1,94280,7413,10469,8790,7924,16735,15987,6619,6101,7826,...,-999999999,7.5,-999999999,8.3,-999999999,8.8,6.4,7.6,06,02310
2,161041,11112,17032,19337,16624,33681,27585,9667,7068,10698,...,-999999999,3.6,-999999999,7.3,-999999999,7.6,4.2,6.1,06,02610
3,146865,13334,16341,16323,16582,24384,22204,9264,8391,13685,...,-999999999,-999999999,-999999999,22.2,-999999999,15.2,9.5,13.2,06,03630
4,64015,2735,4767,6492,6335,9895,11440,5638,5537,6946,...,-999999999,7.5,-999999999,-999999999,-999999999,-999999999,2,6.4,06,04800
5,74714,6671,8885,6438,9792,15099,9882,4613,3847,5782,...,-999999999,-999999999,-999999999,-999999999,-999999999,6,9.3,6.6,06,05580


In [39]:
#Find all columns without "Age" in column title
#[col for col in withoutea.columns if "AGE" not in col]

In [40]:
#Create a new dataframe for data without the columns that have "Age" in the title, which is 35 columns
withoutage = withoutea[[col for col in withoutea.columns if 'AGE' not in col]].copy()

In [41]:
withoutage.head()

,Total!!Estimate!!Population 16 years and over,Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!White alone,Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Black or African American alone,Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!American Indian and Alaska Native alone,Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Asian alone,Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Native Hawaiian and Other Pacific Islander alone,Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Some other race alone,Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Two or more races,Total!!Estimate!!Hispanic or Latino origin (of any race),"Total!!Estimate!!White alone, not Hispanic or Latino",...,Unemployment rate!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!American Indian and Alaska Native alone,Unemployment rate!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Asian alone,Unemployment rate!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Native Hawaiian and Other Pacific Islander alone,Unemployment rate!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Some other race alone,Unemployment rate!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Two or more races,Unemployment rate!!Estimate!!Hispanic or Latino origin (of any race),"Unemployment rate!!Estimate!!White alone, not Hispanic or Latino",Unemployment rate!!Estimate!!Population 20 to 64 years,STATE,SCHOOL DISTRICT (ELEMENTARY)
1,94280,21536,-999999999,-999999999,29605,-999999999,38004,-999999999,54414,7420,...,-999999999,7.5,-999999999,8.3,-999999999,8.8,6.4,7.6,06,02310
2,161041,119572,-999999999,-999999999,17362,-999999999,17639,-999999999,105350,33376,...,-999999999,3.6,-999999999,7.3,-999999999,7.6,4.2,6.1,06,02610
3,146865,114515,10663,-999999999,-999999999,-999999999,13093,-999999999,93833,37706,...,-999999999,-999999999,-999999999,22.2,-999999999,15.2,9.5,13.2,06,03630
4,64015,13018,-999999999,-999999999,42821,-999999999,-999999999,-999999999,-999999999,9767,...,-999999999,7.5,-999999999,-999999999,-999999999,-999999999,2,6.4,06,04800
5,74714,61498,-999999999,-999999999,-999999999,-999999999,-999999999,-999999999,55446,14010,...,-999999999,-999999999,-999999999,-999999999,-999999999,6,9.3,6.6,06,05580


In [42]:
# Format to rename School District Name
# def cleandistrict(NAME):
    # return NAME.replace(", California","")

In [43]:
# Apply formatting condition to School District Name
# withoutage['NAME'].apply(cleandistrict)

In [44]:
# Create new School District name column with formatting titled District
# withoutage['District']= withoutage['NAME'].apply(cleandistrict)

In [45]:
# Find length of Geographt column contents, which is 16
# withoutage['Geography'].apply(len).unique()

# Pull a geography
# geo = "9700000US0622710"

# Find NCESDist ID
# geo.split("US")[1]

In [46]:
# Format to seperate Census Geography code (the state and district ID combined), 
# def splitGeo(geo):
    #return geo.split("US")[1]

In [47]:
# Apply formatting condition to Geography
# withoutage['Geography'].apply(splitGeo)

In [48]:
# Create new Geography name column with formatting titled NCESDist
# withoutage['NCESDist']= withoutage['Geography'].apply(splitGeo)

In [49]:
# Create new column with combined state and district ID
withoutage['NCESDist'] = withoutage['STATE'].astype(str) + withoutage['SCHOOL DISTRICT (ELEMENTARY)']

In [50]:
# Add Year column
withoutage['year']= "2015"

In [51]:
withoutage.head()

,Total!!Estimate!!Population 16 years and over,Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!White alone,Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Black or African American alone,Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!American Indian and Alaska Native alone,Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Asian alone,Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Native Hawaiian and Other Pacific Islander alone,Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Some other race alone,Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Two or more races,Total!!Estimate!!Hispanic or Latino origin (of any race),"Total!!Estimate!!White alone, not Hispanic or Latino",...,Unemployment rate!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Native Hawaiian and Other Pacific Islander alone,Unemployment rate!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Some other race alone,Unemployment rate!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Two or more races,Unemployment rate!!Estimate!!Hispanic or Latino origin (of any race),"Unemployment rate!!Estimate!!White alone, not Hispanic or Latino",Unemployment rate!!Estimate!!Population 20 to 64 years,STATE,SCHOOL DISTRICT (ELEMENTARY),NCESDist,year
1,94280,21536,-999999999,-999999999,29605,-999999999,38004,-999999999,54414,7420,...,-999999999,8.3,-999999999,8.8,6.4,7.6,06,02310,0602310,2015
2,161041,119572,-999999999,-999999999,17362,-999999999,17639,-999999999,105350,33376,...,-999999999,7.3,-999999999,7.6,4.2,6.1,06,02610,0602610,2015
3,146865,114515,10663,-999999999,-999999999,-999999999,13093,-999999999,93833,37706,...,-999999999,22.2,-999999999,15.2,9.5,13.2,06,03630,0603630,2015
4,64015,13018,-999999999,-999999999,42821,-999999999,-999999999,-999999999,-999999999,9767,...,-999999999,-999999999,-999999999,-999999999,2,6.4,06,04800,0604800,2015
5,74714,61498,-999999999,-999999999,-999999999,-999999999,-999999999,-999999999,55446,14010,...,-999999999,-999999999,-999999999,6,9.3,6.6,06,05580,0605580,2015


In [9]:
# Save to csv to proper folder
# withoutage.to_csv("/Users/nataligracia/git/ca-school-enrollment-trend/Elementary2015.csv")

In [53]:
# Rearrange columns in list
# ['NCESDist','District'] + list(without_annotation_df.columns[2:])

In [54]:
# Rearrange columns in dataframe
# final = without_annotation_df[['NCESDist','District','Geography','NAME'] + list(without_annotation_df.columns[2:])]
# final